In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from model import *
import pymongo

connection_string = "<redacted>"

format_data = {"name": "abc"}
format_instance = Format(**format_data)

site_data = {"name": "Abc"}
site_instance = Site(**site_data)

tournament_data = {
    "name": "abc",
    "buyin": 1,
    "guarantee": 2,
    "site": site_instance,
    "formats": [format_instance],
}
tournament_instance = Tournament(**tournament_data)

sessionStats_data = {"winnings": 10, "position": 10, "tournamnets": tournament_instance}
sessionStats_instance = SessionStats(**sessionStats_data)

session_data = {"date": datetime.now(), "length": 4, "stats": sessionStats_instance}
session = Session(**session_data)

myclient = pymongo.MongoClient(connection_string)
mydb = myclient["pokerDb"]
mycol = mydb["sessions"]

x = mycol.insert_one(session)


In [7]:
from model import *
import pymongo
from MongoUtils import *

faker = DataFaker(5, 10, 10000, 100, 2, 5, 10)

mongoFaker = MongoUtils(faker)

mongoFaker.clean_database()

mongoFaker.insert_documents()

mongoFaker.aggregate()

In [5]:
# With references

from MongoUtilsRef import *

faker = DataFaker(5, 10, 10000, 100, 2, 5, 10)

mongoFaker = MongoUtilsRef(faker)

mongoFaker.clean_database()

mongoFaker.insert_documents()

mongoFaker.find_all()

[{'date': datetime.datetime(2021, 9, 30, 0, 0),
  'length': 2,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2017, 1, 30, 0, 0),
  'length': 5,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2008, 2, 21, 0, 0),
  'length': 4,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2000, 3, 19, 0, 0),
  'length': 1,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(1994, 2, 25, 0, 0),
  'length': 4,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2006, 7, 17, 0, 0),
  'length': 1,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2010, 5, 28, 0, 0),
  'length': 1,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2013, 1, 19, 0, 0),
  'length': 4,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(1998, 4, 9, 0, 0),
  'length': 2,
  'tournament': [],
  'session': []},
 {'date': datetime.datetime(2005, 3, 4, 0, 0),
  'length': 1,
  'tournament': [],
  'session